In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalação das dependências necessárias
!pip install unsloth
!pip install transformers datasets accelerate bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.

In [5]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import to_sharegpt, standardize_sharegpt, apply_chat_template, is_bfloat16_supported

# Definindo o caminho do workspace
WORKSPACE_PATH = "/content/drive/MyDrive/fiap/tech-challenge-03/"

# Configurações gerais
model_name = "unsloth/llama-3-8b-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Carregando o modelo e o tokenizador
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Adicionando LoRA ao modelo
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
# Carregando o dataset
dataset = load_dataset('json', data_files={'train': f'{WORKSPACE_PATH}trn.json'})
dataset = dataset['train'].shuffle(seed=42).select(range(10000))

# Verificando a estrutura do dataset
print(dataset.features)
print(dataset[:5])

{'uid': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None), 'target_ind': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'target_rel': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}
{'uid': ['B00004T7CV', '0830752293', '0674991141', 'B004D4WH5A', '0906362431'], 'title': ['Freud 12-178 1-1/8-Inch Diameter by 1-1/2-Inch Double Flute Straight Router Bit with 1/2-Inch Shank', 'Faith Begins @ Home Devotions', 'Lives, XI: Aratus. Artaxerxes. Galba. Otho. General Index (Loeb Classical Library) (Volume XI)', "Super Efficient 3/8&quot; x 50' Stainless Steel Wort Chiller", "The Traveller's Guide to Sacred Ireland: A Guide to the Sacred Places of Ireland, Her Legends, Folklore and People"], 'content': ["These bits cut smoother than other straight bits because of the precise shear and hook angles. End of bit relief allows for fast plunging. Covered by Freud's limited lifetime warranty.", '

In [19]:
print(dataset[90]['title'])
print(dataset[90]['content'])
print(dataset[90]['target_rel'])

How to Start a Vending Business
How to Start A Vending Business provides information and advice about starting a bulk vending route, covering how to buy a route, how to run a route, candy choices, vending machine brands and related topics. Here is what you will discover in this book. How to find cheap vending machines with these 3 simple techniques... 3 little known, yet simple ways to find locations for your vending machines... Secrets of expert bulk vendors that few people ever know about... 3 proven steps to a successful vending route... 2 simple keys (that are right in front of your eyes) to finding the cheapest candy for your route... Warning: 3 thins you should never do when it comes to bulk vending... how to not pay commissions to your vending locations... When to buy a vending route from someone and how to do it... 7 everyday but often overlooked tips nad tricks for getting free locations... How often to change your candy... How to keep track your expenses... The once famous bu

In [7]:
# Preparando os dados usando to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt="Título do produto: {title}",
    output_column_name="content",  # Usando 'content' como a coluna de saída
    conversation_extension=3,
)
dataset = standardize_sharegpt(dataset)

Merging columns:   0%|          | 0/10000 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/10000 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10000 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10000 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10000 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/10000 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
# Definindo o template de chat corrigido
chat_template = """Responda à pergunta sobre o produto com base no título e na descrição fornecidos.

### Título do produto:
{INPUT}

### Pergunta:
Quais são as principais características deste produto?

### Resposta (baseada na descrição):
{OUTPUT}

### Título do produto:
{INPUT}

### Pergunta:
Quais são as principais características deste produto?

### Resposta (baseada na descrição):
{OUTPUT}"""

# Aplicando o template de chat
dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
# Configurando o treinamento
training_args = TrainingArguments(
    output_dir=f'{WORKSPACE_PATH}results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    max_steps=30,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none",
)

# Configurando o SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
# Treinamento
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.238000
20,1.980300
30,1.857300


TrainOutput(global_step=30, training_loss=2.0251684188842773, metrics={'train_runtime': 484.3116, 'train_samples_per_second': 0.496, 'train_steps_per_second': 0.062, 'total_flos': 6513610959814656.0, 'train_loss': 2.0251684188842773, 'epoch': 0.024})

In [13]:
# Salvando o modelo
trainer.save_model(f"{WORKSPACE_PATH}unsloth_model_optimized")
tokenizer.save_pretrained(f"{WORKSPACE_PATH}unsloth_model_optimized")

('/content/drive/MyDrive/fiap/tech-challenge-03/unsloth_model_optimized/tokenizer_config.json',
 '/content/drive/MyDrive/fiap/tech-challenge-03/unsloth_model_optimized/special_tokens_map.json',
 '/content/drive/MyDrive/fiap/tech-challenge-03/unsloth_model_optimized/tokenizer.json')

In [14]:
# Função para gerar respostas
def generate_response(title, question):
    FastLanguageModel.for_inference(model)  # Habilita inferência 2x mais rápida

    messages = [
        {"role": "user", "content": f"Título do produto: {title}\n\nPergunta: {question}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer, skip_prompt=True)

    output = model.generate(
        input_ids,
        streamer=text_streamer,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Resposta (baseada na descrição):')[-1].strip()

    return response

# Exemplo de uso
title = "BodyBio/E-Lyte - Chlorella 300 tab"
question = "Quais são as principais características deste produto?"
response = generate_response(title, question)
print(f"Título do produto: {title}")
print(f"Pergunta: {question}")
print(f"Resposta gerada: {response}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Chlorella is a single cell algae that grows in fresh water. It is a rich source of protein and other nutrients. Chlorella is a good source of chlorophyll, which is important for the body.<|end_of_text|>
Título do produto: BodyBio/E-Lyte - Chlorella 300 tab
Pergunta: Quais são as principais características deste produto?
Resposta gerada: Chlorella is a single cell algae that grows in fresh water. It is a rich source of protein and other nutrients. Chlorella is a good source of chlorophyll, which is important for the body.


In [ ]:
# Exemplo de uso
title = "How to Start a Vending Business"
question = "Quais são as principais características deste produto?"
response = generate_response(title, question)
print(f"Título do produto: {title}")
print(f"Pergunta: {question}")
print(f"Resposta gerada: {response}")

The vending machine industry is growing, and there is a huge opportunity for small businesses to get into the market. The vending business is a great way to make money, and this book will teach you how to start a vending business and run it successfully. This book will teach you how to start a vending business and run it successfully. It will teach you how to choose the right products to sell, how to find the best locations for your machines, and how to 